<img src="data/photutils_banner.svg" width=500 alt="Photutils logo" style="margin-left: 0;">

<div class="alert alert-block alert-info">
<h2 style="margin-top: 0">In this notebook, we will cover:</h2>

- The basics to perform aperture photometry with the `aperture_photometry` function
- Aperture photometry (and other properties) with the `ApertureStats` class

This notebook builds on the previous tutorials for Astropy Units/Quantities, Coordinates, FITS, and Tables.
</div>

## Preliminaries

In [ ]:
# Initial imports
import numpy as np
import matplotlib.pyplot as plt

# Change some default plotting parameters
import matplotlib as mpl
mpl.rcParams['image.origin'] = 'lower'
mpl.rcParams['image.interpolation'] = 'nearest'

# Run the %matplotlib magic command to enable inline plotting
# in the current notebook.  Choose one of these:
%matplotlib inline
# %matplotlib notebook

### Load the data

We'll start by reading science data and error arrays from FITS files located in the [**data/**](data) subdirectory.  The FITS files contain 2D cutout images from the [Hubble Extreme-Deep Field (XDF)](https://archive.stsci.edu/prepds/xdf/) taken with the [Wide Field Camera 3 (WFC3)](https://www.stsci.edu/hst/instrumentation/wfc3) IR channel in the F160W filter (centered at ~1.6 $\mu m$).

In [ ]:
from astropy.io import fits

sci_fn = 'data/xdf_hst_wfc3ir_60mas_f160w_sci.fits'
rms_fn = 'data/xdf_hst_wfc3ir_60mas_f160w_rms.fits'
sci_hdulist = fits.open(sci_fn)
rms_hdulist = fits.open(rms_fn)

Let's print some information about the science data.

In [ ]:
sci_hdulist.info()

The data array is in the **`0`** extension, has a shape of `200 x 200` pixels, and has a data type of `float32` (32 bit floating-point numbers).

Let's extract the data and error arrays.

In [ ]:
data = sci_hdulist[0].data.astype(float)
error = rms_hdulist[0].data.astype(float)

`data` and `error` are now 2D [numpy](https://numpy.org) arrays, each with a shape of 200 x 200 pixels.

In [ ]:
data.shape, error.shape

Now let's extract the data header and create an Astropy [World Coordinate System (WCS)](https://docs.astropy.org/en/stable/wcs/index.html) object from the FITS header WCS information.

In [ ]:
from astropy.wcs import WCS

hdr = sci_hdulist[0].header
wcs = WCS(hdr)

Finally, let's display the science image.  Here we use the [astropy.visualization](https://docs.astropy.org/en/stable/visualization/index.html) subpackage to apply a square-root stretch to the data.

In [ ]:
from astropy.visualization import simple_norm

plt.figure(figsize=(5, 5))
norm = simple_norm(data, 'sqrt', percent=99.0)
plt.imshow(data, norm=norm)
plt.title('HST F160W XDF Cutout');

# Aperture Objects

Photutils provides circular, elliptical, and rectangular aperture shapes (plus annulus versions of each).

Further, there are two types of aperture classes, defined either with pixel or sky (celestial) coordinates.

These are the names of the [aperture classes](https://photutils.readthedocs.io/en/latest/aperture.html#classes) that are defined in pixel coordinates:

* `CircularAperture`
* `CircularAnnulus`

* `EllipticalAperture`
* `EllipticalAnnulus`

* `RectangularAperture`
* `RectangularAnnulus`

The aperture classes defined in celestial coordinates have `Sky` prepended to their names:

* `SkyCircularAperture`
* `SkyCircularAnnulus`

* `SkyEllipticalAperture`
* `SkyEllipticalAnnulus`

* `SkyRectangularAperture`
* `SkyRectangularAnnulus`


The aperture shapes look like this:

<img src='data/apertures.png' alt='Figure of aperture shapes' width=700px>

## Methods for handling aperture/pixel overlap

The apertures will usually only partially overlap some of the pixels in the data.

Photutils apertures provide three methods for handling the aperture overlap with the pixel grid of the data array.

<img src="data/photutils_aperture_methods.svg">

For the default method (`method='exact'`), the exact fractional overlap of the aperture with each pixel is calculated. Each pixel value is then weighted by the exact fractional overlap of the aperture.

For `method='center'`, a pixel is considered to be entirely in or out of the aperture depending on whether its center is located within the aperture.

For `method='subpixel'`, pixels are divided into a number of subpixels, which are in or out of the aperture based on their centers. Each pixel value is then weighted by its fraction of subpixel overlaps. For this method, the number of subpixels needs to be set with the `subpixels` keyword. The `subpixels` keyword is ignored for the `'exact'` and `'center'` methods.

## Creating Aperture Objects

First, let's define a circular aperture at a given (x, y) pixel position and radius (in pixels).

Photutils has many tools for detecting sources (e.g., see [photutils.detection](https://photutils.readthedocs.io/en/latest/detection.html) and [photutils.segmentation](https://photutils.readthedocs.io/en/latest/segmentation.html)) and measuring source positions (e.g., [photutils.centroids](https://photutils.readthedocs.io/en/latest/centroids.html)). For this example, we'll assume we've already measured the source centroid.

In [ ]:
from photutils.aperture import CircularAperture

position = (90.73, 59.43)  # (x, y) centroid position (pixels)
radius = 5.0  # pixels
aperture = CircularAperture(position, r=radius)

Our `aperture` variable is now an instance of a [CircularAperture](https://photutils.readthedocs.io/en/latest/api/photutils.aperture.CircularAperture.html#photutils.aperture.CircularAperture) object.

In [ ]:
aperture

We can plot the aperture on the data using its `plot` method, with a custom color and line width.

In [ ]:
plt.figure(figsize=(5, 5))
plt.imshow(data, norm=norm)
aperture.plot(color='red', lw=2);

# Performing Aperture Photometry

Now let's perform photometry on the XDF data using this circular aperture.

After the aperture object is created, we can  perform the photometry using the [aperture_photometry](https://photutils.readthedocs.io/en/latest/api/photutils.aperture.aperture_photometry.html#photutils.aperture.aperture_photometry) function or the [ApertureStats](https://photutils.readthedocs.io/en/latest/api/photutils.aperture.ApertureStats.html#photutils.aperture.ApertureStats) class.  We'll start by using the `aperture_photometry` function.  The `ApertureStats` class will be introduced later in this notebook.

In the most basic case, we simply need to input the data array and the aperture object. The default aperture overlap method is `'exact'`.

Note that the input data array is assumed to be background subtracted.  If that is not the case, please see the documentation for the [photutils.background](https://photutils.readthedocs.io/en/latest/background.html) subpackage for tools to help subtract the background.

The background has already been subtracted from our XDF example data, so we skip that step in this example.

<div class="alert alert-warning alert-block">
<h3 style="margin-top:0;">Learn More</h3>

See the [local background subtraction notebook](03-aperture_local_bkgsub.ipynb) for examples of aperture photometry with local background subtraction.
</div>

In [ ]:
from photutils.aperture import aperture_photometry

phot = aperture_photometry(data, aperture)
phot

The output is an Astropy `QTable` (Quantity Table) with several columns.  The sum of the data values within the circular aperture is in the `aperture_sum` column.

The table also contains metadata, which is accessed by the `meta` attribute of the table.  The metadata is stored in a python dictionary.

In [ ]:
phot.meta

In [ ]:
phot.meta['aperture_photometry_args']

The other aperture overlap methods are specified using the `method` keyword.  Aperture photometry using the **'center'** method gives a slightly less accurate answer.

In [ ]:
phot = aperture_photometry(data, aperture, method='center')
phot

Now perform aperture photometry using the `'subpixel'` method with `subpixels=5`:

These parameters are equivalent to SourceExtractor aperture photometry.

In [ ]:
phot = aperture_photometry(data, aperture, method='subpixel', subpixels=5)
phot

## Photometric Errors

We can also input an error array to get the photometric errors.

In [ ]:
phot = aperture_photometry(data, aperture, error=error)
phot

The error keyword expects the **total** error array (i.e., all sources of error).

However, the error array in our XDF FITS file represents only the background error.  Thus, we need to also include the source Poisson error as follows:

$\sigma_{\mathrm{tot}} = \sqrt{\sigma_{\mathrm{b}}^2 + \frac{I}{g}}$
  
where $\sigma_{\mathrm{tot}}$ is the total error (in the same units as the data), $\sigma_{\mathrm{b}}$ is the background-only error, $I$ are the data values, and $g$ is the "effective gain".

The "effective gain" is the value (or an array if it's variable across an image) needed to convert the data image to count units (e.g., electrons or photons), where Poisson statistics apply.

Photutils provides a [calc_total_error](https://photutils.readthedocs.io/en/stable/api/photutils.utils.calc_total_error.html) function to perform the calculation of combining background-only error with the source Poisson error.

In [ ]:
# this time include the Poisson error of the source
from photutils.utils import calc_total_error

# our data array is in units of e-/s
# so the "effective gain" should be the exposure time
eff_gain = hdr['TEXPTIME']  # exposure time from the FITS header
total_error = calc_total_error(data, error, eff_gain)

phot = aperture_photometry(data, aperture, error=total_error)
phot

The aperture photometry error increased only slightly because this is a small faint source.

## Units

`Quantity` inputs for the `data` and `error` arrays are also allowed.  Note that the unit must be the same for the `data` and `error` inputs.

The XDF dataset has units of electron/s. One way to check this is to look at the value of the `BUNIT` FITS header keyword.

In [ ]:
hdr['BUNIT']

In [ ]:
import astropy.units as u

unit = u.electron / u.s  # unit for the data and error arrays
phot = aperture_photometry(data << unit, aperture, error=total_error << unit)
phot

The table columns now contain `Quantity` arrays.

In [ ]:
phot['aperture_sum']

## Performing aperture photometry for multiple sources

Now let's perform aperture photometry for three sources (all with the same aperture size).  We simply define three (x, y) positions.

In [ ]:
positions = [(90.73, 59.43), (73.63, 139.41), (43.62, 61.63)]
radius = 5.0  # pixels
apertures = CircularAperture(positions, r=radius)

In [ ]:
apertures

Note that an aperture object with multiple positions can be indexed or sliced to get a subset of apertures:

In [ ]:
apertures[1]  # aperture for the second position

In [ ]:
apertures[0:2]  # apertures for the first two positions

Let's plot all three apertures on the data.

In [ ]:
plt.figure(figsize=(5, 5))
plt.imshow(data, norm=norm)
apertures.plot(color='red', lw=2);

Now let's perform aperture photometry for these three sources.

In [ ]:
phot = aperture_photometry(data << unit, apertures, error=total_error << unit)
phot

Each source is a row in the table and is given a unique **`id`** number (the first column).

Since we input arrays with units, the output columns are `Quantity` arrays.

In [ ]:
phot['aperture_sum']

## Adding columns to the photometry table

Let's calculate the signal-to-noise (SNR) ratio of our sources and add it as a new column to the table.

In [ ]:
snr = phot['aperture_sum'] / phot['aperture_sum_err']  # units will cancel
phot['snr'] = snr
phot

As mentioned above, this dataset was taken with the HST WFC3/IR F160W filter.  Using the [F160W zero point](https://www.stsci.edu/hst/instrumentation/wfc3/data-analysis/photometric-calibration/ir-photometric-calibration), let's calculate the F160W AB magnitude and add it as a new column in the table.

In [ ]:
f160w_zpt = 25.9463  # HST/WFC3 F160W ABmag zero point for flux in e-/s

# NOTE that the log10 function can be applied only to dimensionless quantities,
# so we use the value attribute for Quantity objects to remove the units of the aperture sum
abmag = -2.5 * np.log10(phot['aperture_sum'].value) + f160w_zpt

phot['f160w_abmag'] = abmag
phot

Using the `WCS` transform defined when we loaded the data, we can also calculate the sky coordinates for these objects and add them to the table.

In [ ]:
# convert pixel positions to sky coordinates
x, y = np.transpose(positions)
sky_coord = wcs.pixel_to_world(x, y)  # a SkyCoord object

# we can add the astropy SkyCoord object directly to the table
phot['sky_coord'] = sky_coord
phot

In [ ]:
phot['sky_coord']  # a SkyCoord object

## Saving a photometry table

If we write the table to an ASCII file using the ECSV format, we can read it back in later preserving all of the table metadata and `Quantity` and `SkyCoord` objects.

In [ ]:
aper_filename = 'f160w_aperture_photometry.ecsv'
phot.write(aper_filename, overwrite=True)

Now let's read the table into a new variable.

In [ ]:
from astropy.table import QTable

tbl = QTable.read(aper_filename)
tbl

In [ ]:
# the table metadata was preserved
tbl.meta

In [ ]:
# as was the Quantity arrays, e.g.:
tbl['aperture_sum']

In [ ]:
# and the SkyCoord array
tbl['sky_coord']

## Aperture photometry using Sky apertures

Aperture photometry can also be performed using `SkyCoord` positions instead of pixel positions.

First, let's define the sky coordinates by converting our pixel coordinates using our WCS transform.

In [ ]:
positions = [(90.73, 59.43), (73.63, 139.41), (43.62, 61.63)]
x, y = np.transpose(positions)
coord = wcs.pixel_to_world(x, y)
coord

Now let's define circular apertures centered at these sky coordinates.

For sky apertures, the aperture radius must be a `Quantity` in angular units.

In [ ]:
from photutils.aperture import SkyCircularAperture

radius = 0.5 * u.arcsec  # angular units
sky_apers = SkyCircularAperture(coord, r=radius)
sky_apers.r

When performing photometry using a sky aperture, `aperture_photometry` needs the WCS transformation, which is input via the `wcs` keyword.

In [ ]:
phot_sky = aperture_photometry(data, sky_apers, wcs=wcs)
phot_sky

# ApertureStats

The [ApertureStats](https://photutils.readthedocs.io/en/latest/api/photutils.aperture.ApertureStats.html#photutils.aperture.ApertureStats) class can be used to create a catalog of statistics and properties for pixels within an aperture, including aperture photometry. 

It can calculate many properties, including statistics like `min`, `max`, `mean`, `median`, `std`, `sum_aper_area`, and `sum`. It also can be used to calculate morphological properties like `centroid`, `fwhm`, `semimajor_sigma`, `semiminor_sigma`, `orientation`, and `eccentricity`. Please see `ApertureStats` for the the complete list of properties that can be calculated. The properties can be accessed using `ApertureStats` attributes or output to an Astropy `QTable` using the `to_table` method.

Most of the source properties are calculated using the "center" aperture-mask method, which gives aperture weights of 0 or 1. This avoids the need to compute weighted statistics for partial pixel overlaps -- the data pixel values are directly used.

The `sum_method` and `subpixels` keywords are used to determine the aperture-mask method when calculating the sum-related properties: `sum`, `sum_error`, `sum_aper_area`, `data_sumcutout`, and `error_sumcutout`. The default is `sum_method='exact'`, which produces exact aperture-weighted photometry.

The optional `local_bkg` keyword can be used to input the per-pixel local background of each source, which will be subtracted before computing the aperture statistics.

The optional `sigma_clip` keyword can be used to sigma clip the pixel values before computing the source properties. This keyword could be used, for example, to compute a sigma-clipped median of pixels in an annulus aperture to estimate the local background level.

Note that like `aperture_photometry`, `ApertureStats` expects the input data to be background subtracted.  The background has already been subtracted from our XDF example data, so we skip that step in this example.

As before, we start by creating an `Aperture` object.

In [ ]:
positions = [(90.73, 59.43), (73.63, 139.41), (43.62, 61.63)]
radius = 5.0  # pixels
apertures = CircularAperture(positions, r=radius)

Now let's create an `ApertureStats` object.

In [ ]:
from photutils.aperture import ApertureStats

apstats = ApertureStats(data << unit, apertures, error=total_error << unit)
apstats

The source properties can be accessed as attributes, for example the source sum (and errors) for aperture photometry.

In [ ]:
apstats.sum

In [ ]:
apstats.sum_err

Note that these values are exactly the same as those calculated earlier using `aperture_photometry`.

In [ ]:
phot['aperture_sum'], phot['aperture_sum_err']

Going further, here are the calculated source centroids.

In [ ]:
apstats.xcentroid, apstats.ycentroid

All of the source properties can be output as an astropy `QTable` using the `to_table` method.

In [ ]:
apstats.to_table()

# Exercise

Measure the circular-aperture fluxes (and errors) of the three sources centered at (x, y) positions of `(34.61, 137.99), (184.53, 157.52), and (88.75, 184.22)` within a radius of `4.5` pixels.

Calculate their signal-to-noise ratios and F160W AB magnitudes.

Plot the apertures on the data.

Hints:

* The HST/WFC3 F160W ABmag zero point for data in units of e-/s is `25.9463`.

If you prefer to load the solution, for each cell below uncomment the cell and **run it twice** (once to load the solution and once more to run it).

In [ ]:
# %load 01-aperture_exercise_solution

In [ ]:
# %load 01-aperture_exercise_solution_plot

<div class="alert alert-warning alert-block">
<h3 style='margin-top: 0;'>Learn More</h3>

The [aperture photometry extended notebook](02-aperture_extended.ipynb) covers:

- Aperture Photometry
    - Performing aperture photometry at multiple positions with the same aperture
      - Using the aperture_photometry function
      - Using the ApertureStats class
    - Performing aperture photometry at multiple positions using different apertures
      - Using the aperture_photometry function
      - Using the ApertureStats class
    - Bad pixel masking
      - Using the aperture_photometry function
      - Using the ApertureStats class
    - Encircled flux
      - Using the aperture_photometry function
      - Using the ApertureStats class

- ApertureMask objects
</div>